Welcome! We first import the needed packages.

In [11]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

## 0. Goal

The goal of this part of the project is to make a linear model to predict death rates in different states and countries. 

# Preparing Data

## 1. Loading the data

#1 Let's first load the data and clean it a little.

In [35]:
original_data1 = pd.read_csv('4.18states.csv')
original_data1_cleaned = original_data1.set_index('Province_State')
#print(original_data1_cleaned.columns)
#print(original_data1_cleaned.index)
#original_data1_cleaned

In [37]:
original_data2 = pd.read_csv('abridged_couties.csv')
#print(original_data2.columns)
original_data2

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,POP_LONGITUDE,...,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,-86.494165,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4354,NaN,NaN,NaN
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,-87.762381,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,-85.310038,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.9959,6.08,5400.0,18241.0
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,-87.127659,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.6003,2.75,14980.0,6120.0
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,-86.591491,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4242,7.21,31850.0,25233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,15005,15.0,5.0,Kalawao,HI,NaN,NaN,NaN,21.188495,-156.979972,...,737509.0,737507.0,737504.0,737509.0,737500.0,737495.0,0.3162,NaN,NaN,NaN
3240,72039,72.0,39.0,Ciales Municipio,PR,NaN,NaN,NaN,18.314399,-66.494215,...,NaN,737500.0,737499.0,737499.0,737500.0,737495.0,NaN,NaN,NaN,NaN
3241,72069,72.0,69.0,Humacao Municipio,PR,NaN,NaN,NaN,18.144804,-65.817109,...,NaN,737500.0,737499.0,737499.0,737500.0,737495.0,NaN,NaN,NaN,NaN
3242,City1,NaN,NaN,New York City,NY,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To begin with, we first take an "educated guess" and select a few features of interest in the second table, which we think will affect the death rate of Covid 19. 


In [38]:
operate_data2 = original_data2[['CountyName', 'State', 'lat', 'stay at home', 'public schools', '>500 gatherings'
                                 ]] 
operate_data2

,CountyName,State,lat,stay at home,public schools,>500 gatherings
0,Autauga,Alabama,32.540091,737519.0,737500.0,737497.0
1,Baldwin,Alabama,30.738314,737519.0,737500.0,737497.0
2,Barbour,Alabama,31.874030,737519.0,737500.0,737497.0
3,Bibb,Alabama,32.999024,737519.0,737500.0,737497.0
4,Blount,Alabama,33.990440,737519.0,737500.0,737497.0
...,...,...,...,...,...,...
3239,Kalawao,NaN,NaN,737509.0,737507.0,737509.0
3240,Ciales Municipio,NaN,NaN,NaN,737500.0,NaN
3241,Humacao Municipio,NaN,NaN,NaN,737500.0,NaN
3242,New York City,NaN,NaN,NaN,NaN,NaN


We find the mortality rate in the first (aggregate) table, then append it to each entry in the second table, based on the State of each entry.

**Question**: can you take the aggregate data from a bigger granularity (state) to a data of lower granularity (county)? The mortality rate in different county is different, and since each county has different population, we cannot generalize one state's mortality rate to all counties in it?

**For now** we will just use all other factors to *falsely* predict #latitude, and change it back to mortality rate later. I am not sure how to append mortality rates to counties (temporarily lost my memory) but will fix it later. I still attached my starter code below.

In [39]:
mortality_rate = original_data1_cleaned['Mortality_Rate']
#operate_data2['mortality_rate'] = operate_data2[operate_data2['State'].isin(mortality_rate.index)]
mortality_rate


Province_State
Alabama           3.247029
Alaska            2.866242
American Samoa         NaN
Arizona           3.810330
Arkansas          2.178899
                    ...   
Xinjiang          3.947368
Yukon             0.000000
Yunnan            1.086957
Zhejiang          0.078864
Grand Princess    0.000000
Name: Mortality_Rate, Length: 140, dtype: float64

# EDA

## 2. Missing Values

#2 Now we will look at missing values.

In [16]:
operate_data2.isnull().sum()

CountyName           0
State              169
lat                169
stay at home       592
public schools      25
>500 gatherings    221
dtype: int64

We can first disregard the counties without a state (but need to investigate which counties they are in later). 

In [17]:
# counties WITHOUT states.
#operate_data2[operate_data2['State'].isnull()]


In [31]:
operate_data2_with_states = operate_data2[operate_data2['State'].notnull()]
#operate_data2_with_states


We should replace the NaN values of 'stay at home', 'public school', and '>500 gatherings' with the average value in their states. 

**For Now** for simplicity I will replace all values in the three columns with an average value of the whole table (which is incorrect but convenient). 

**Question** not sure why the error pops up, also this code looks pretty ugly.

In [33]:
operate_data2_with_states['stay at home'] = operate_data2_with_states['stay at home'].fillna(np.mean(operate_data2_with_states['stay at home']))
operate_data2_with_states['public schools'] = operate_data2_with_states['public schools'].fillna(np.mean(operate_data2_with_states['public schools']))
operate_data2_with_states['>500 gatherings'] = operate_data2_with_states['>500 gatherings'].fillna(np.mean(operate_data2_with_states['>500 gatherings']))
operate_data2_with_states.isnull().sum()
#operate_data2_with_states[operate_data2_with_states['stay at home'].isnull()]

/srv/conda/envs/data100/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/srv/conda/envs/data100/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/data100/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

CountyName         0
State              0
lat                0
stay at home       0
public schools     0
>500 gatherings    0
dtype: int64

Now we can see all the null values are filled.

## 3 Outliers

#3 Let's observe whether there is any outliers. 

In [20]:
operate_data2_with_states['stay at home'].value_counts()
operate_data2_with_states['public schools'].value_counts()
operate_data2_with_states['>500 gatherings'].value_counts()

737501.000000    474
737497.000000    309
737508.000000    268
737498.000000    254
737509.000000    236
737502.000000    197
737510.000000    195
737503.000000    151
737496.000000    129
737503.761164    119
737505.000000    115
737517.000000     99
737499.000000     88
737511.000000     87
737513.000000     77
737507.000000     75
737500.000000     62
737512.000000     56
737495.000000     39
737504.000000     23
737514.000000     22
Name: >500 gatherings, dtype: int64

By observing the value_counts, we find that there is no significant outlier. 

**To Do:** Here we can also draw some scatter plot to show the relation between some features we choose and the response vector (death rate), and show they have positive correlation.

## 4. Train-Test Split

#4 Let's do a train-test split.

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(operate_data2_with_states, test_size=0.1, random_state=42)

Let's do some feature selection. What are some good features to use to predict death rates? 

# Classification

## 5. Basic Classification

#5 After selecting features, now we will make training set and test sets. 

In [22]:
X_train = train.drop(['lat', 'CountyName', 'State'], axis=1)
Y_train = train['lat']

X_train[:5], Y_train[:5]

(       stay at home  public schools  >500 gatherings
 3033  737508.000000        737500.0         737508.0
 477   737518.000000        737502.0         737501.0
 1688  737512.476209        737503.0         737503.0
 1880  737506.000000        737502.0         737497.0
 2689  737517.000000        737507.0         737498.0, 3033    38.775217
 477     33.483357
 1688    41.910133
 1880    41.890001
 2689    30.317938
 Name: lat, dtype: float64)

Now we will make a linear regression model, fit the model with training data, and get the training loss. 

Loss function: I will use RMSE now but we can try other loss functions later. 

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics



model = LinearRegression(fit_intercept=True) # should fit intercept be true?
model.fit(X_train, Y_train)

Y_prediction = model.predict(X_train)


training_loss = metrics.mean_squared_error(Y_prediction, Y_train)
print("Training loss: ", training_loss)

Training loss:  16.412422919778123


## 6. Cross-Validation

#6 Now let's perform cross-validation to check whether we are overfitting, and determine which features are useful. 

In [26]:
# perform cross validation
from sklearn import model_selection as ms

# finding which features to use using Cross Validation
errors = []
range_of_num_features = range(1, X_train.shape[1] + 1)
for N in range_of_num_features:
    print(f"Trying first {N} features")
    model = LinearRegression()
    
    # compute the cross validation error
    error = ms.cross_val_score(model, X_train.iloc[:, 0:N], Y_train).mean()
    
    print("\tScore:", error)
    errors.append(error)

best_num_features = np.argmax(errors) + 1
print (best_num_features)
best_err = min(errors)

print(f"Best choice, use the first {best_num_features} features")

Trying first 1 features
	Score: 0.23284529538706739
Trying first 2 features
	Score: 0.2323168254291003
Trying first 3 features
	Score: 0.3003127658312975
3
Best choice, use the first 3 features


## 7. Regularization

#7 Here we perform regularization of data.

## 8. Test

#8 Here we will test our model on test set and find its test accuracy. 

## 9. Conclusion

#9 We can also fit other models (like logistic model?) with our training data, and test on test set about their accuracy.